In [1]:
from torch.utils.data import Dataset
import torch
import numpy as np
from spectral import open_image
import pickle

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [3]:
from torch.utils.data import Dataset
import torch
import numpy as np
from spectral import open_image
import pickle
import cv2
import rasterio

In [4]:
class numpyDataset:
    """returns bigger image you need to split them while training."""

    def __init__(self, file_paths, std_scaler_path=None,normalization_par = None):
        """
        Args:
            file_paths (list): list of filepaths of HDR files .
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file_IDs = file_paths # HDR file path
        self.normalization_par = normalization_par
        if normalization_par == None:
            dbfile = open(std_scaler_path, 'rb')
            self.standardisation_param = pickle.load(dbfile)

    def __len__(self):
        return len(self.file_IDs)

    def getitem(self, index: int):
        filepath = self.file_IDs[index]
        image_sample = rasterio.open(str(filepath), "r")
        bands = [i for i in range(1, image_sample.count+1)]
        image_sample = image_sample.read(bands)
        image_sample = image_sample.astype('float32')
        
        img_std = image_sample/5500
        
        #img_std = np.expand_dims(img_std, axis=0)

        img_tensor = torch.from_numpy(img_std)
        

        sample = (img_tensor, 0)

        return sample

In [5]:
list_of_best_images = [
    '../input/eurosat-dataset/EuroSATallBands/AnnualCrop/AnnualCrop_1026.tif',
    '../input/eurosat-dataset/EuroSATallBands/HerbaceousVegetation/HerbaceousVegetation_1032.tif',
    '../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1012.tif',
    '../input/eurosat-dataset/EuroSATallBands/Industrial/Industrial_10.tif',
    '../input/eurosat-dataset/EuroSATallBands/Pasture/Pasture_1044.tif',
    '../input/eurosat-dataset/EuroSATallBands/PermanentCrop/PermanentCrop_1013.tif',
    '../input/eurosat-dataset/EuroSATallBands/River/River_1004.tif',
    '../input/eurosat-dataset/EuroSATallBands/SeaLake/SeaLake_1002.tif',
    '../input/eurosat-dataset/EuroSATallBands/AnnualCrop/AnnualCrop_1064.tif',
    '../input/eurosat-dataset/EuroSATallBands/HerbaceousVegetation/HerbaceousVegetation_100.tif',
    '../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1034.tif',
    '../input/eurosat-dataset/EuroSATallBands/Industrial/Industrial_1017.tif',
    '../input/eurosat-dataset/EuroSATallBands/Pasture/Pasture_1068.tif',
    '../input/eurosat-dataset/EuroSATallBands/PermanentCrop/PermanentCrop_1073.tif',
    '../input/eurosat-dataset/EuroSATallBands/River/River_103.tif',
    '../input/eurosat-dataset/EuroSATallBands/SeaLake/SeaLake_1072.tif'
]

In [6]:
len(list_of_best_images)

In [7]:
DL = numpyDataset(list_of_best_images, normalization_par = 5500)

In [8]:
DL.__len__()

In [9]:
test_tensor_vis = []
for i in range(16):
    test_tensor_vis.append(DL.getitem(i)[0])

In [11]:
test_tensor_vis = torch.stack(test_tensor_vis).to(device)

In [12]:
torch.save(test_tensor_vis, 'test_tensor_vis.pt')

In [13]:
test_tensor_vis.shape

In [3]:
import os
os.listdir('../input/eurosat-dataset/EuroSATallBands/SeaLake')[-1]

In [ ]:
test_tensor_vis

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
import torch

class logger_regression:
    def __init__(self,test_tensor_path,device):
        self.losslistinstance_train = []
        self.losslistinstance_train_cont = []
        self.epochwiselosstrain = []

        self.losslistinstance_val = []
        self.losslistinstance_val_cont = []
        self.epochwiselossval = []

        self.mselistinstance_val = []
        self.epochwisemseval = []

        self.psnrlistinstance_val = []
        self.epochwisepsnrval = []

        self.test_tensor = torch.load(test_tensor_path)
        self.device = device

        self.LRS = []
        self.imagenumber = 0
        if os.path.exists('visuals'):
            pass
        else:
            os.mkdir('visuals')

    def updateLossListTrain(self,loss,LRC):
        self.losslistinstance_train.append(loss)
        self.LRS.append(LRC)

    def updateLossListval(self,lossdict):
        loss = lossdict['loss']
        mse = lossdict['mse']
        psnr = lossdict['psnr']
        self.losslistinstance_val.append(loss)
        self.mselistinstance_val.append(mse)
        self.psnrlistinstance_val.append(psnr)

    def displayLossTrain(self):
        avg_loss = np.array(self.losslistinstance_train).mean()
        print("Train: Average Squared Recunstruction Loss:",avg_loss)

    def displayLossval(self):
        avg_loss = np.array(self.losslistinstance_val).mean()
        avg_MSE = np.array(self.mselistinstance_val).mean()
        avg_PSNR = np.array(self.psnrlistinstance_val).mean()
        print("Val: Average Squared Recunstruction Loss:",avg_loss)
        print("Val: Average MSE between original images and Recunstructed images:",avg_MSE)
        print("Val: Average PSNR between original images and Recunstructed images:", avg_PSNR)
        return avg_loss

    def resetTrainLoss(self):
        avg_loss = np.array(self.losslistinstance_train).mean()
        self.epochwiselosstrain.append(avg_loss)
        self.losslistinstance_train_cont.extend(self.losslistinstance_train)
        self.losslistinstance_train = []

    def resetvalLoss(self):
        avg_loss = np.array(self.losslistinstance_val).mean()
        avg_MSE = np.array(self.mselistinstance_val).mean()
        avg_PSNR = np.array(self.psnrlistinstance_val).mean()

        self.epochwiselossval.append(avg_loss)
        self.epochwisemseval.append(avg_MSE)
        self.epochwisepsnrval.append(avg_PSNR)

        self.losslistinstance_val_cont.extend(self.losslistinstance_val)

        self.losslistinstance_val = []
        self.mselistinstance_val = []
        self.psnrlistinstance_val = []


    def plotLoss(self):
        plt.plot(self.epochwiselosstrain, '-bx')
        plt.plot(self.epochwiselossval, '-rx')
        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.legend(['Training', 'Validation'])
        plt.title('Loss vs. No. of epochs')
        plt.savefig('visuals/0_lossplot.png')
        plt.show()

        plt.plot(self.epochwisemseval, '-bx')
        plt.xlabel('epochs')
        plt.ylabel('MSE')
        plt.title('Val: MSE vs. No. of Epochs')
        plt.savefig('visuals/0_mse.png')
        plt.show()

        plt.plot(self.epochwisepsnrval, '-bx')
        plt.xlabel('epochs')
        plt.ylabel('PSNR (in dB)')
        plt.title('Val: PSNR vs. No. of Epochs')
        plt.savefig('visuals/0_psnr.png')
        plt.show()

        plt.plot(self.losslistinstance_train_cont, '-bx')
        plt.xlabel('Training Steps')
        plt.ylabel('Loss')
        plt.title('Train: Loss vs. No. of Steps')
        plt.savefig('visuals/0_train_cont_loss.png')
        plt.show()

        plt.plot(self.losslistinstance_val_cont, '-bx')
        plt.xlabel('Validation Steps')
        plt.ylabel('Loss')
        plt.title('Val: Loss vs. No. of Steps')
        plt.savefig('visuals/0_val_cont_loss.png')
        plt.show()

    def plotLR(self):
        plt.plot(self.LRS)
        plt.xlabel('Batch no.')
        plt.ylabel('Learning rate')
        plt.title('Learning Rate vs. Batch no.')
        plt.savefig('visuals/0_LRplot.png')
        plt.show()

    def visualize_io(self,model):
        o = model(self.test_tensor.to(self.device))

        im_arr_ip = []
        for i in range(8):
            im_arr_ip.append(self.test_tensor.cpu().numpy()[i,2, :, :])

        for i in range(8):
            im_arr_ip.append(o.cpu().detach().numpy()[i,2, :, :])

        fig = plt.figure(figsize=(20., 5.))

        grid = ImageGrid(fig, 111,  # similar to subplot(111)
                         nrows_ncols=(2, 8),  # creates 2x2 grid of axes
                         axes_pad=0.1,  # pad between axes in inch.
                         )

        for ax, im in zip(grid, im_arr_ip):
            # Iterating over the grid returns the Axes.
            ax.imshow(im, cmap='gray')

        plt.savefig('visuals/io_' + str(self.imagenumber) + '.png')
        self.imagenumber = self.imagenumber + 1
        plt.show()

    def savemodel(self,model,modelpath):
        return None

In [16]:
import torch.nn as nn
import torch.nn.functional as F

#Encoder V3 Dataset V2

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()
        self.leakyreLU = nn.LeakyReLU()
        
        #encoder
        
        self.conv1_1 = nn.Conv2d(in_channels=13, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv1_2= nn.Conv2d(in_channels=32, out_channels=32, kernel_size=1, stride=2, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(32)
        
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2_2= nn.Conv2d(in_channels=32, out_channels=32, kernel_size=1, stride=2, padding=0)
        self.batch_norm2 = nn.BatchNorm2d(32)
        
        self.conv3_1 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv3_2= nn.Conv2d(in_channels=16, out_channels=16, kernel_size=1, stride=2, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(16)
        
        self.conv4_1 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv4_2= nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1, stride=2, padding=0)
        self.batch_norm4 = nn.BatchNorm2d(8)
        
        
        # Decoder
        
        self.convT1_1 = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=1, stride=2,output_padding=1)
        self.convT1_2=nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=3, stride=1,output_padding=0,padding=1)
        self.batch_norm5 = nn.BatchNorm2d(16)
        
        self.convT2_1 = nn.ConvTranspose2d(in_channels=16, out_channels=16, kernel_size=1, stride=2,output_padding=1)
        self.convT2_2=nn.ConvTranspose2d(in_channels=16, out_channels=32, kernel_size=3, stride=1,output_padding=0,padding=1)
        self.batch_norm6 = nn.BatchNorm2d(32)
        
        self.convT3_1 = nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=1, stride=2,output_padding=1)
        self.convT3_2= nn.ConvTranspose2d(in_channels=32, out_channels=64, kernel_size=3, stride=1,output_padding=0,padding=1)
        self.batch_norm7 = nn.BatchNorm2d(64)
        
        self.convT4_1 = nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=1, stride=2,output_padding=1)
        self.convT4_2= nn.ConvTranspose2d(in_channels=32, out_channels=13, kernel_size=3, stride=1,output_padding=0,padding=1)


    def forward(self, x):
        x = self.leakyreLU(self.batch_norm1(self.conv1_2(self.conv1_1(x))))
        #x = self.leakyreLU(self.batch_norm2(self.conv2_2(self.conv2_1(x))))
        #x = self.leakyreLU(self.batch_norm3(self.conv3_2(self.conv3_1(x))))
        #x = self.leakyreLU(self.batch_norm4(self.conv4_2(self.conv4_1(x))))

        #x = self.leakyreLU(self.batch_norm5(self.convT1_2(self.convT1_1(x))))
        #x = self.leakyreLU(self.batch_norm6(self.convT2_2(self.convT2_1(x))))
        #x = self.leakyreLU(self.batch_norm7(self.convT3_2(self.convT3_1(x))))
        x = self.sigmoid(self.convT4_2(self.convT4_1(x)))
        return x


model = Net()

In [19]:
model = model.to(device)

In [26]:
logger = logger_regression('test_tensor_vis.pt', device)

In [27]:
logger.visualize_io(model)